In [17]:
from pyspark.sql import SparkSession

In [18]:
spark_server = "spark://spark-master-otmzsp:7077"
minio_server = "http://minio-otmzsp:9000" 
data_inicio_processamento = "2024-09-23 21:00:00"

In [19]:
spark = SparkSession.builder \
    .appName("Processa Posições Trusted") \
    .master(spark_server) \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0,org.apache.hadoop:hadoop-aws:3.3.4,org.elasticsearch:elasticsearch-spark-30_2.12:7.17.24") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.endpoint", minio_server) \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("es.nodes.wan.only", "true") \
    .config("es.nodes", "elasticsearch-otmzsp:9200") \
    .getOrCreate()

In [20]:
spark

In [21]:
trusted = 's3a://trusted/posicoes'
checkpoint = 's3a://trusted/posicoes_checkpoint_leitura'

In [22]:
try:
    from elasticsearch import Elasticsearch
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "elasticsearch"])
    from elasticsearch import Elasticsearch


In [23]:
# Configurações
ES_URL = "http://elasticsearch-otmzsp:9200"  # URL do Elasticsearch
KIBANA_URL = "http://kibana-otmzsp:5601"
# Nome index pattern
INDEX_NAME = "posicoes_full"  # Nome do índice
INDEX_PATTERN_NAME = INDEX_NAME#+"-*"

In [24]:
import requests

# Conectar ao Elasticsearch
es = Elasticsearch(ES_URL)  # ajuste a URL se necessário

# Verificar se o índice existe
if not es.indices.exists(index=INDEX_NAME):
    # Mapeamento do índice
    mapping = {
                "mappings": {
                    "properties": {
                    "id": {
                        "type": "keyword"
                    },
                    "veiculo_horario_referencia": {
                        "type": "date"
                    },
                    "veiculo_letreiro_completo": {
                        "type": "text",
                        "fields": {
                        "keyword": {
                            "type": "keyword",
                            "ignore_above": 256
                        }
                        }
                    },
                    "veiculo_linha_codigo": {
                        "type": "text",
                        "fields": {
                        "keyword": {
                            "type": "keyword",
                            "ignore_above": 256
                        }
                        }
                    },
                    "veiculo_acessibilidade": {
                        "type": "text",
                        "fields": {
                        "keyword": {
                            "type": "keyword",
                            "ignore_above": 256
                        }
                        }
                    },
                    "veiculo_sentido": {
                        "type": "text",
                        "fields": {
                        "keyword": {
                            "type": "keyword",
                            "ignore_above": 256
                        }
                        }
                    },
                    "veiculo_letreiro_destino": {
                        "type": "text",
                        "fields": {
                        "keyword": {
                            "type": "keyword",
                            "ignore_above": 256
                        }
                        }
                    },
                    "veiculo_letreiro_origem": {
                        "type": "text",
                        "fields": {
                        "keyword": {
                            "type": "keyword",
                            "ignore_above": 256
                        }
                        }
                    },
                    "qtde_veiculos_linha": {
                        "type": "long"
                    },
                    "veiculo_prefixo": {
                        "type": "text",
                        "fields": {
                        "keyword": {
                            "type": "keyword",
                            "ignore_above": 256
                        }
                        }
                    },
                    "veiculo_horario_utc_captura": {
                        "type": "date"
                    },
                    "veiculo_latitude": {
                        "type": "float"
                    },
                    "veiculo_longitude": {
                        "type": "float"
                    },
                    "veiculo_horario_local_captura": {
                        "type": "date"
                    },
                    "tipo_operacao_linha": {
                        "type": "text"
                    },
                    "position": {
                        "type": "geo_point"
                    },
                    "Prefixo Veículo": {
                        "type": "alias",
                        "path": "veiculo_prefixo"
                    },
                    "Letreiro": {
                        "type": "alias",
                        "path": "veiculo_letreiro_completo"
                    },
                    "Origem": {
                        "type": "alias",
                        "path": "veiculo_letreiro_origem"
                    },
                    "Destino": {
                        "type": "alias",
                        "path": "veiculo_letreiro_destino"
                    },
                    "Sentido": {
                        "type": "alias",
                        "path": "veiculo_sentido"
                    },
                    "Acessibilidade": {
                        "type": "alias",
                        "path": "veiculo_acessibilidade"
                    },
                    "Data/Hora": {
                        "type": "alias",
                        "path": "veiculo_horario_utc_captura"
                    }
                }
            }
        }


    # Criar o índice com o mapeamento
    es.indices.create(index=INDEX_NAME, body=mapping)
    print(f"Índice '{INDEX_NAME}' criado com sucesso.")
else:
    print(f"O índice '{INDEX_NAME}' já existe.")



O índice 'posicoes_full' já existe.


In [25]:
TIME_FIELD_NAME = "veiculo_horario_local_captura" 

# Verificar se o padrão de índice existe no Kibana
headers = {"kbn-xsrf": "true"}
response = requests.get(f"{KIBANA_URL}/api/saved_objects/_find?type=index-pattern&search_fields=title&search={INDEX_PATTERN_NAME}", headers=headers)

if response.status_code == 200:
    data = response.json()
    if len(data['saved_objects']) == 0:
        # Criar o padrão de índice se não existir
        payload = {
            "attributes": {
                "title": INDEX_PATTERN_NAME,
                "timeFieldName": TIME_FIELD_NAME
            }
        }
        create_response = requests.post(f"{KIBANA_URL}/api/saved_objects/index-pattern", json=payload, headers=headers)

        if create_response.status_code == 200:
            print(f"Padrão de índice '{INDEX_PATTERN_NAME}' criado com sucesso.")
        else:
            print(f"Erro ao criar padrão de índice: {create_response.text}")
    else:
        print(f"Padrão de índice '{INDEX_PATTERN_NAME}' já existe.")
else:
    print(f"Erro ao verificar padrão de índice: {response.text}")

Padrão de índice 'posicoes_full' já existe.


In [26]:
from datetime import datetime, timedelta, timezone

def generate_hourly_partitions(start_date_str):
    # Converte a string de data e hora para um objeto datetime
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M:%S")
    
    # Define o fuso horário GMT-3
    gmt_minus_3 = timezone(timedelta(hours=-3))
    
    # Obtém a hora atual em GMT-3 menos 1 hora
    now = datetime.now(gmt_minus_3)
    
    # Lista para armazenar as partições
    partitions = []
    
    # Itera desde a data de início até a hora atual menos 1 hora
    current = start_date.replace(tzinfo=gmt_minus_3)
    while current <= now:
        partition = f"datepartition={current.strftime('%Y-%m-%d-%H')}"
        partitions.append(partition)
        current += timedelta(hours=1)
    
    return partitions


partitions = generate_hourly_partitions(data_inicio_processamento)
print(partitions)

['datepartition=2024-09-23-21']


In [27]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, FloatType, TimestampType

# Definindo o esquema manualmente
schema = StructType([
    StructField("veiculo_horario_referencia", TimestampType(), True),
    StructField("veiculo_letreiro_completo", StringType(), True),
    StructField("veiculo_linha_codigo", StringType(), True),
    StructField("veiculo_sentido", StringType(), True),
    StructField("veiculo_letreiro_destino", StringType(), True),
    StructField("veiculo_letreiro_origem", StringType(), True),
    StructField("qtde_veiculos_linha", LongType(), True),
    StructField("veiculo_prefixo", LongType(), True),
    StructField("veiculo_acessibilidade", StringType(), True),
    StructField("veiculo_horario_utc_captura", TimestampType(), True),
    StructField("veiculo_latitude", FloatType(), True),
    StructField("veiculo_longitude", FloatType(), True),
    StructField("veiculo_horario_local_captura", TimestampType(), True),
    StructField("tipo_operacao_linha", StringType(), True),
    StructField("id", StringType(), True)
])


# Agora pode seguir com o processamento


In [29]:
from pyspark.sql.utils import AnalysisException
from pyspark.sql import functions as F

for datepartition in partitions:
    try:
        # Criar o DataFrame de streaming
        df_posicoes = spark.read \
            .option("header", "true") \
            .schema(schema) \
            .option("ignoreNullFields", "true") \
            .csv(trusted + f'/{datepartition}')
        #df_posicoes.show()

        if not df_posicoes.isEmpty():
            df_posicoes = df_posicoes.withColumn(
                    "position", 
                    F.concat(F.col("veiculo_latitude").cast("string"), F.lit(","), F.col("veiculo_longitude").cast("string"))
                )

            df_posicoes.write \
                .format("org.elasticsearch.spark.sql") \
                .option("es.net.nodes", ES_URL) \
                .option("es.mapping.id", "id") \
                .mode("append") \
                .save(INDEX_NAME)
            print(f'Escreveu no Elastic! {datepartition}')
    except AnalysisException as e:
        print(f"Erro ao salvar o arquivo: {e}")

Erro ao salvar o arquivo: [PATH_NOT_FOUND] Path does not exist: s3a://trusted/posicoes/datepartition=2024-09-23-21.


In [32]:
spark.stop()